<a href="https://colab.research.google.com/github/shamimkhaled/mindfulness-meditation-agent/blob/main/Mindfulness_Meditation_Agent_to_Generate_Structured_Outputs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install the necessary libraries

In [39]:
!pip install pydantic pytest openai==0.28

# Define the Pydantic Model

In [40]:
from pydantic import BaseModel, Field
from typing import List

class Phrase(BaseModel):
    text: str
    pause: int  # milliseconds

class MeditationScript(BaseModel):
    duration: int  # milliseconds
    focus_area: str
    phrases: List[Phrase]


# Implement the Mediatation Agent to Generate Structured Outputs

In [42]:
import openai
from getpass import getpass

# Securely input your OpenAI API key
api_key = getpass("Enter your OpenAI API key: ")
openai.api_key = api_key


Enter your OpenAI API key: ··········


In [43]:
# List available models
models = openai.Model.list()
print([model['id'] for model in models['data']])

['chatgpt-4o-latest', 'dall-e-2', 'gpt-4o-2024-08-06', 'gpt-4-turbo-preview', 'gpt-4o', 'gpt-3.5-turbo-instruct', 'tts-1', 'tts-1-1106', 'gpt-4-0125-preview', 'gpt-3.5-turbo-0125', 'gpt-3.5-turbo', 'babbage-002', 'davinci-002', 'dall-e-3', 'gpt-4-turbo-2024-04-09', 'tts-1-hd', 'tts-1-hd-1106', 'text-embedding-ada-002', 'gpt-3.5-turbo-16k', 'gpt-4-1106-preview', 'gpt-4o-realtime-preview-2024-10-01', 'text-embedding-3-small', 'whisper-1', 'text-embedding-3-large', 'gpt-4-turbo', 'gpt-4o-2024-05-13', 'gpt-3.5-turbo-1106', 'gpt-4-0613', 'gpt-4', 'gpt-3.5-turbo-instruct-0914', 'gpt-4o-mini', 'gpt-4o-realtime-preview', 'gpt-4o-mini-2024-07-18']


In [44]:
        # Simulate structured data (you can dynamically parse it)
        # structured_outputs = {
        #     "duration": 45000,
        #     "focus_area": "relaxation",
        #     "phrases": [
        #         {"text": "Welcome to this meditation, take a deep breath in through your nose and out through your mouth.", "pause": 2000},
        #         {"text": "Feel the air fill your lungs, and then release any tension or stress as you exhale.", "pause": 2000},
        #         {"text": "Bring your attention to your body, starting from your toes and moving up to the top of your head.", "pause": 2000},
        #         {"text": "As you breathe in, imagine fresh, calming energy entering your body.", "pause": 2000},
        #         {"text": "As you breathe out, imagine any tension or stress leaving your body.", "pause": 2000},
        #         {"text": "Allow your muscles to relax, starting from your toes and moving up to the top of your head.", "pause": 4000},
        #         {"text": "Feel the weight of your body sinking into the ground, supported by the earth below.", "pause": 3000},
        #         {"text": "Imagine yourself in a peaceful, safe place, surrounded by calm and serenity.", "pause": 4000},
        #         {"text": "Stay here for a moment, breathing deeply and feeling the relaxation spread throughout your body.", "pause": 6000},
        #         {"text": "When you're ready, slowly open your eyes, and take a deep breath in, feeling refreshed and renewed.", "pause": 2000}
        #     ]
        # }

In [45]:
def convert_to_milliseconds(meditation_script: dict) -> dict:
    """
    Convert the duration and pause times in the meditation script from seconds to milliseconds.
    """
    # Convert duration from seconds to milliseconds
    meditation_script["duration"] = meditation_script["duration"] * 1000

    # Convert each phrase's pause time from seconds to milliseconds
    for phrase in meditation_script["phrases"]:
        phrase["pause"] = phrase["pause"] * 1000

    return meditation_script

In [46]:

import openai
from pydantic import ValidationError
# from models import MeditationScript
import json
# Set your OpenAI API key
# openai.api_key = 'your-openai-api-key-here'

def generate_meditation_script(prompt: str) -> MeditationScript:
    try:


        # Check if the prompt is empty or invalid
        if not prompt.strip():
            print("Invalid prompt: The prompt cannot be empty.")
            return None

        # Define the schema for the structured output
        response_format = {
            "type": "json_schema",
            "json_schema": {
                "name": "meditation_response",
                "schema": {
                    "type": "object",
                    "properties": {
                        "duration": {"type": "integer"},
                        "focus_area": {"type": "string"},
                        "phrases": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "text": {"type": "string"},
                                    "pause": {"type": "integer"}
                                },
                                "required": ["text", "pause"],
                                "additionalProperties": False
                            }
                        }
                    },
                    "required": ["duration", "focus_area", "phrases"],
                    "additionalProperties": False
                },
                "strict": True
            }
        }

        # Call OpenAI API to generate a response based on the prompt
        response = openai.ChatCompletion.create(
            model="gpt-4o-2024-08-06",
            messages=[
                {"role": "system", "content": "You are a mindfulness meditation assistant that provides structured outputs. Your role is to create clear and concise mindfulness meditation scripts, tailored to the user's request. Please ensure the output follows the structure of duration, focus_area, and phrases with pauses."},
                {"role": "user", "content": prompt},
            ],
            response_format=response_format,
            max_tokens=500,
            temperature=0.7
        )

        # Extract the generated text
        generated_text = response.choices[0].message['content']



        # print(f"Generated Text: {generated_text}")  # Debug: Print the raw output

        # Attempt to parse the JSON-like response into a structured format
        structured_output = json.loads(generated_text)

          # Convert duration and pause times to milliseconds
        converted_output = convert_to_milliseconds(structured_output)



        # Validate the structured data with Pydantic
        meditation_script = MeditationScript(**converted_output)
        return meditation_script

    except ValidationError as e:
        print(f"Validation Error: {e}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


# Test the meditation script

In [47]:
# Define your prompt for the meditation script
prompt = "can you make a 60 second mindfulness meditation?"

# Generate the meditation script
meditation_script = generate_meditation_script(prompt)

# Display the structured meditation script
if meditation_script:
    print(meditation_script.model_dump_json(indent=2))
else:
    print("Failed to generate meditation script.")

{
  "duration": 60000,
  "focus_area": "breath awareness",
  "phrases": [
    {
      "text": "Find a comfortable seated position and gently close your eyes.",
      "pause": 5000
    },
    {
      "text": "Take a deep breath in through your nose, feeling your chest and belly expand.",
      "pause": 10000
    },
    {
      "text": "Hold the breath for a moment... and now slowly exhale through your mouth.",
      "pause": 10000
    },
    {
      "text": "With each breath, allow yourself to become more present in this moment.",
      "pause": 10000
    },
    {
      "text": "Notice the sensation of the air as it enters and leaves your body.",
      "pause": 10000
    },
    {
      "text": "If your mind wanders, gently bring your focus back to your breath.",
      "pause": 10000
    },
    {
      "text": "Take one more deep, calming breath in... and slowly release it.",
      "pause": 5000
    }
  ]
}


In [48]:
# Define your prompt for the meditation script
prompt = "Create a mindfulness meditation script focused on relaxation."

# Generate the meditation script
meditation_script = generate_meditation_script(prompt)

# Display the structured meditation script
if meditation_script:
    print(meditation_script.model_dump_json(indent=2))
else:
    print("Failed to generate meditation script.")

{
  "duration": 10000,
  "focus_area": "relaxation",
  "phrases": [
    {
      "text": "Find a comfortable position, either sitting or lying down.",
      "pause": 5000
    },
    {
      "text": "Gently close your eyes and take a deep breath in through your nose.",
      "pause": 5000
    },
    {
      "text": "Hold that breath for just a moment.",
      "pause": 3000
    },
    {
      "text": "Now, slowly exhale through your mouth, releasing any tension.",
      "pause": 5000
    },
    {
      "text": "Bring your attention to your breath, noticing the natural rhythm.",
      "pause": 5000
    },
    {
      "text": "Feel the rise and fall of your chest with each inhale and exhale.",
      "pause": 5000
    },
    {
      "text": "With each breath, imagine tension melting away from your body.",
      "pause": 5000
    },
    {
      "text": "Notice your shoulders drop and your muscles soften.",
      "pause": 5000
    },
    {
      "text": "Visualize a warm, calming light spreadi